In [1]:
from typing import Dict, Optional
from sortedcontainers import SortedDict  # type: ignore
import os

import ipysheet as sheet
from IPython.display import display, HTML
import mammoth

from const import IDX_COL, STYLE_COL
from config import FROM_LANG, TO_LANG
from setup import sl_sem, gr_sem

from util import ord_word
from semantics import TableSemantics

from merger import merge
from aggregator import aggregate
from exporter import export_docx
from generator import generate_docx


In [2]:
sla = SortedDict(ord_word)
gre = SortedDict(ord_word)

pairs = [
    TableSemantics(
        sl_sem, gr_sem, label="от славянски основен към гръцки", result=sla
    ),
    TableSemantics(
        sl_sem.var, gr_sem, label="от славянски вариант към гръцки", result=sla
    ),
    TableSemantics(
        gr_sem, sl_sem, label="от гръцки основен към славянски", result=gre
    ),
    TableSemantics(
        gr_sem.var, sl_sem, label="от гръцки вариант към славянски", result=gre
    ),
]
sem = pairs[0]



In [3]:
def _style2str(s: Dict[str,str], bgs: Dict[str, Optional[str]]) -> str:
    """take font style from index and presence of background in selected named columns"""
    result = [k for k, v in bgs.items() if v]
    if s and s['fontWeight'] == 'bold':
        result.append("bold")
    if s and s['fontStyle'] == 'italic':
        result.append("italic")
    return "|".join(result)


In [4]:

r1 = (
    ["вѣдѣл\ue205 WG оувѣдѣл\ue205 H", "ѹвѣдѣт\ue205 H"]
    + [""] * 2
    + ["12/67c10", "в\ue205дѣл\ue205", "в\ue205дѣл\ue205 бꙑхо-", "в\ue205дѣт\ue205"]
    + [""] * 3
    + ["ἔγνωμεν", "γιγνώσκω"]
    + [""] * 13
)
r2 = (
    [""] * 4
    + ["12/67c10", "бꙑхомъ•", "в\ue205дѣл\ue205 бꙑхо-", "бꙑт\ue205"]
    + ["", "gramm.", ""] * 2 + [""]* 12
)
rows = (r1, r2)
input = sheet.sheet(rows=len(rows), columns=STYLE_COL)
# styled = sheet.cell(1, 9, background_color="lightblue")
# rows = sheet.cell_range(rows)
cells = [[sheet.cell(r,c, rows[r][c], background_color='lightblue' if r==1 and c==9 else None) for c, vc in enumerate(r1)] for r, vr in enumerate(rows)]
input

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='вѣдѣл\ue205 WG оув…

In [5]:
lines = []
blank = False
for row in cells:
    line = [cell.value.strip() if cell.value else cell.value for cell in row]
    # Two consequent blank lines
    if blank and not [l for l in line if l]:
        break
    irow = [v for v in row]
    bgs = {f"hl{v:02d}": irow[v].style['backgroundColor'] for v in sem.cols() if 'backgroundColor' in irow[v].style}
    line.append(_style2str(row[IDX_COL].style, bgs))
    lines.append(line)
    blank = not [l for l in line if l]
print(f"{len(lines)} думи")

2 думи


In [6]:
for p in pairs:
    print(f"Събиране на многоредови преводи {p.label}...")
    merged = merge(lines, p.orig, p.trans)
    print(f"{len(merged)} думи")

    print(f"Кондензиране {p.label}...")
    before = len(p.result)
    p.result = aggregate(merged, p.orig, p.trans, p.result)
    after = len(p.result)
    print(f"{after-before} леми")


Събиране на многоредови преводи от славянски основен към гръцки...
2 думи
Кондензиране от славянски основен към гръцки...
2 леми
Събиране на многоредови преводи от славянски вариант към гръцки...
2 думи
Кондензиране от славянски вариант към гръцки...
1 леми
Събиране на многоредови преводи от гръцки основен към славянски...
2 думи
Кондензиране от гръцки основен към славянски...
2 леми
Събиране на многоредови преводи от гръцки вариант към славянски...
2 думи
Кондензиране от гръцки вариант към славянски...
0 леми


In [7]:
export_fname = "temp.docx"


In [8]:
print("Генериране славянски...")
generate_docx(sla, FROM_LANG, export_fname)
with open(export_fname, "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file)
    print(result.messages) # Any messages, such as warnings during conversion
    display(HTML(result.value))    

Генериране славянски...
[]


In [9]:
print("Генериране гръцки...")
generate_docx(gre, TO_LANG, export_fname)
with open(export_fname, "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file)
    print(result.messages) # Any messages, such as warnings during conversion
    display(HTML(result.value))    

Генериране гръцки...
[]


In [10]:
print("Експорт славянски...")
export_docx(sla, FROM_LANG, export_fname)
with open(export_fname, "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file)
    print(result.messages) # Any messages, such as warnings during conversion
    display(HTML(result.value))    

Експорт славянски...
[]


In [11]:
print("Експорт гръцки...")
export_docx(gre, TO_LANG, export_fname)
with open(export_fname, "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file)
    print(result.messages) # Any messages, such as warnings during conversion
    display(HTML(result.value))    

Експорт гръцки...
[]


In [12]:
os.remove(export_fname)